## 1-Verify system(python intepreter) and python version number)

In [1]:
import sys #provide objects/functions interact with intepretwer
print (sys.version) # python 2.7 is needed to run Keras Reinforcement Learning(Environment OpenAIGYM)

3.5.3 |Continuum Analytics, Inc.| (default, May 15 2017, 10:43:23) [MSC v.1900 64 bit (AMD64)]


In [2]:
#magic set up matplotlib to work interactively(within Jnote)
%matplotlib inline 

## 2-Load dependencies(interactive data science and machine learning-Keras reinforcement learning) 

In [3]:
import matplotlib.pyplot as plt #provides a MATLAB like plotting framework(Jnote)
import numpy as np #provide computing array(carry to tensorflow) format 
import pandas as pd #provide data analysis and manipulation Python library
import seaborn as sns #provide statistical date visualization(linear+distribution+timeseries)
sns.set() #set seaborn format(font, color, axes styple) in one step
import keras #provide tensorflow and framework(17 Keras libraries)
import numpy as np
import gym

from keras.models import Sequential #provide linear stack of layers to create neural network
from keras.layers import Dense #provide dot product function,  output = activation(dot(input, kernel) + bias)` 
from keras.layers import Activation #provide activation(etc, RELU) function to the output layer 
from keras.layers import Flatten #flatten the input(etc, 60,60,3) becomes 10,800
from keras.optimizers import Adam #provide Adam optimzer(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0, **kwargs)
from rl.agents.dqn import DQNAgent #take in agent policy, provides compile(take in optimizer, metric) function
from rl.policy import BoltzmannQPolicy # available options for EpsGreedyPolicy
from rl.memory import SequentialMemory #provide data ring bufferbuffer rather than deque data structure

Using TensorFlow backend.


## Build environment(openai gym Cartpole) and neural network(3 dense layers- 16 neurons/layer)

In [4]:
ENV_NAME = 'CartPole-v0' # Acrobot-v1, MountainCar-v0, is functional in Win10(supports python35 upwards)

# Get the environment and extract the number of actions.
env = gym.make(ENV_NAME)  #provide the environment with step, reset, render
np.random.seed(123)
env.seed(123) #sets the the seed for this env's random number generator
nb_actions = env.action_space.n # equal nb_actions with the environment action state(etc cart pole has 2 states, move horizontally left or right)

# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(16))
model.add(Activation('relu'))
model.add(Dense(nb_actions))
model.add(Activation('linear'))
print(model.summary())

[2017-07-20 16:57:08,495] Making new env: CartPole-v0


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_1 (Flatten)          (None, 4)                 0         
_________________________________________________________________
dense_1 (Dense)              (None, 16)                80        
_________________________________________________________________
activation_1 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_2 (Activation)    (None, 16)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 16)                272       
_________________________________________________________________
activation_3 (Activation)    (None, 16)                0         
__________

## Train the network using Deep Q Network learning with Boltzmann policy, and Adam optimizer

In [5]:
# Finally, we configure and compile our agent. You can use every built-in Keras optimizer and
# even the metrics!
memory = SequentialMemory(limit=50000, window_length=1) #set memory for network training 
policy = BoltzmannQPolicy() #set the policy, agent takes in default(EpsGreedy) as the default policy 
dqn = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=10,
               target_model_update=1e-2, policy=policy)
dqn.compile(Adam(lr=1e-3), metrics=['mae'])
#dqn.load_weights('dqn_cartPole-v0_weights.h5f')
# Okay, now it's time to learn something! We visualize the training here for show, but this
# slows down training quite a lot. You can always safely abort the training prematurely using
# Ctrl + C.
dqn.fit(env, nb_steps=10, visualize=True, verbose=2) #take in environment, number of training step-1 training steps takes average 0.3sec, turn on 
#training visualization

# After training is done, we save the final weights
dqn.save_weights('dqn_{}_weights.h5f'.format('CartPole-weights-after-100actions'), overwrite=True)

Training for 10 steps ...
done, took 0.368 seconds


## Test the Deep Q Network with results(rewards) over 5 episode

In [6]:
result = dqn.test(env, nb_episodes=5, visualize=True)

Testing for 5 episodes ...
Episode 1: reward: 58.000, steps: 58
Episode 2: reward: 25.000, steps: 25
Episode 3: reward: 149.000, steps: 149
Episode 4: reward: 20.000, steps: 20
Episode 5: reward: 86.000, steps: 86
